# Imports

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
import pickle
import os

In [ ]:
from djimaging.user.alpha.schemas.alpha_schema import *
from djimaging.user.alpha.utils import populate_alpha

populate_alpha.load_alpha_config(schema_name=populate_alpha.SCHEMA_PREFIX + "glu")
populate_alpha.load_alpha_schema(create_schema=True, create_tables=True)

In [ ]:
schema

# ERD

In [ ]:
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore", FutureWarning)
    display(dj.ERD(schema))

# Core

In [ ]:
if len(Experiment().proj()) == 0:
    populate_alpha.populate_experiments(verbose=True)

In [ ]:
populate_alpha.populate_core(verbose=True)

# Morphology

## Cell positions

In [ ]:
populate_alpha.populate_cell_positions()

In [ ]:
RetinalFieldLocationCat().plot('nt_side')
RetinalFieldLocationWing().plot()

## Linestacks

In [ ]:
populate_alpha.populate_morphology(verbose=True)

In [ ]:
populate_alpha.populate_additional_morph_metrics(verbose=True)

In [ ]:
ConvexHull().plot1();

## Make sure field stamps are consistent

In [ ]:
field_keys = (Field & (RoiKind & 'roi_kind="roi"')).fetch('KEY')

for field_key in field_keys:
    f_xcoord_um, f_ycoord_um, f_zcoord_um = (Field & field_key).fetch('absx', 'absy', 'absz')
    xcoord_ums, ycoord_ums, zcoord_ums = (Presentation.ScanInfo() & field_key).fetch('xcoord_um', 'ycoord_um', 'zcoord_um')

    dists = ((xcoord_ums-f_xcoord_um)**2 + (ycoord_ums-f_ycoord_um)**2 + (zcoord_ums-f_zcoord_um)**2)**0.5

    if np.any(dists > 10):
        print(field_key)
        print(dists)
        print('x:', xcoord_ums-f_xcoord_um)
        print('y:', ycoord_ums-f_ycoord_um)
        print('z:', zcoord_ums-f_zcoord_um)
        print()

# Match fields

## Load Match templates

In [ ]:
from djimaging.utils.mask_utils import to_igor_format

root = os.path.abspath('../../../../../../data/Ran/sONa_glutamate/match_templates/')

for key in MatchTemplate().key_source:
    key = (Field & key).fetch1('KEY')
    fromfile = (Field & key).fetch1('fromfile')
    
    try:
        pres_key = (Presentation & dict(h5_header=fromfile)).fetch1('KEY')
    except:
        continue
        
    filepath = "---".join([str(key['date']).replace('-', ''), str(key['exp_num']), 'Pre', os.path.basename(fromfile).replace('.h5', '') + '_template.pkl'])
    print(filepath)
    assert os.path.isfile(os.path.join(root, filepath)), filepath

    with open(os.path.join(root, filepath), 'rb') as f:
        match_template = pickle.load(f)

    MatchTemplate().insert1({**pres_key, "roi_mask": to_igor_format(match_template)}, skip_duplicates=True)

## Fit templates to Linestacks

In [ ]:
populate_alpha.populate_fit_to_morphology(verbose=True)

In [ ]:
(Field & {'experimenter': 'Ran', 'date': '20201127', 'exp_num': 1, 'field': 'd4'}).delete()

## Auto fit

In [ ]:
populate_alpha.populate_fit_to_morphology(verbose=True, processes=20)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 3))
sns.histplot(ax=axs[0], data=FieldStackPos.fetch(format='frame'), x='rec_cpos_stack_fit_dist')
sns.histplot(ax=axs[1], data=FieldStackPos.FitInfo.fetch(format='frame'), x='score')
plt.show()

### Plot potential mismatches

In [ ]:
from djimaging.utils.dj_utils import get_primary_key

keys = [get_primary_key(FieldStackPos(), key=row) for row in (FieldStackPos.FitInfo() & "score<-3")]

for key in keys:
    print(key)
    FieldStackPos().plot1(key=key)

In [ ]:
for row in (FieldStackPos() & "rec_cpos_stack_fit_dist>50"):
    print(row)
    FieldStackPos().plot1(key=get_primary_key(FieldStackPos(), key=row))

In [ ]:
FieldPosMetrics().plot1(key={'experimenter': 'Ran', 'date': "20201111", 'exp_num': 1, 'field': 'd5'})

# Field ROIs

Field ROIs are simply all ROIs as one

In [ ]:
populate_alpha.add_field_rois(verbose=True);

In [ ]:
FieldRoiPosMetrics().populate(display_progress=True, processes=1)

In [ ]:
RoiKind().populate(display_progress=True, processes=20)
RoiKind()

In [ ]:
plt.hist(RoiKind().fetch('roi_kind'));

In [ ]:
# Populate core after adding Field ROIs
populate_alpha.populate_core(verbose=True)

# Responses

## Surround index

In [ ]:
SineSpotSurroundIndex().populate(processes=20, display_progress=True)

In [ ]:
(Snippets & dict(stim_name='sinespot')).plot1()

In [ ]:
SineSpotSurroundIndex().plot1()

In [ ]:
plt.hist(SineSpotSurroundIndex().fetch('sinespot_surround_index'), bins=51);

In [ ]:
ChirpSurroundIndex().populate(processes=20, display_progress=True)

In [ ]:
ChirpSurroundIndex().plot1()

In [ ]:
plt.hist(ChirpSurroundIndex().fetch('chirp_surround_index'), bins=51);

## Quality

In [ ]:
populate_alpha.populate_quality(verbose=True)

In [ ]:
QualityParams()

In [ ]:
(ChirpQI() & (RoiKind & "roi_kind='roi'")).plot()

In [ ]:
(ChirpQI() & (RoiKind & "roi_kind='field'")).plot()

## Metrics

In [ ]:
populate_alpha.populate_metrics(verbose=True)

# Convex Hull

In [ ]:
ConvexHull().plot1()

In [ ]:
df_convex_hull = (ConvexHull() * RetinalFieldLocationCat().proj(group="nt_side")).fetch(format='frame')
df_convex_hull.head()

In [ ]:
df_convex_hull.to_csv('data/convex_hull_glutamate.csv')